# 

# What I did:

-  First I checked the null values, instead removing the null values I used imputation methods.
-  I generated one unique account number for 224 null values in account number column.

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load the dataset
data = pd.read_csv('simulated_transaction_2024.csv')
data.head()

,Date,Timestamp,Account No,Balance,Amount,Third Party Account No,Third Party Name
0,01/01/2023,00:00,678330503.0,2971.0,1584.0,NaN,Westport Care Home
1,01/01/2023,00:00,472213568.0,3792.0,1950.0,NaN,Barbiee Boutique
2,01/01/2023,00:00,472213568.0,3012.0,-780.0,283027736.0,NaN
3,01/01/2023,00:00,283027736.0,1787.0,780.0,472213568.0,NaN
4,01/01/2023,00:00,624500124.0,3226.0,1825.0,NaN,Fat Face


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230596 entries, 0 to 230595
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Date                    230367 non-null  object 
 1   Timestamp               230345 non-null  object 
 2   Account No              230372 non-null  float64
 3   Balance                 230350 non-null  float64
 4   Amount                  230387 non-null  float64
 5   Third Party Account No  6832 non-null    float64
 6   Third Party Name        223517 non-null  object 
dtypes: float64(4), object(3)
memory usage: 12.3+ MB


In [3]:
data.shape

(230596, 7)

In [4]:
data.columns

Index(['Date', 'Timestamp', 'Account No', 'Balance', 'Amount',
       'Third Party Account No', 'Third Party Name'],
      dtype='object')

In [5]:
data.isnull().sum()

Date                         229
Timestamp                    251
Account No                   224
Balance                      246
Amount                       209
Third Party Account No    223764
Third Party Name            7079
dtype: int64

In [6]:
# Grouping the dataset by 'Account No' and counting the number of transactions for each account
transactions_per_user = data.groupby('Account No').size()

# Converting the series to a DataFrame for better readability
transactions_per_user_df = transactions_per_user.reset_index(name='Number of Transactions')

# Displaying the first few rows to verify
transactions_per_user_df.head()

,Account No,Number of Transactions
0,101531259.0,123
1,104832000.0,164
2,105375973.0,158
3,106601471.0,193
4,108481285.0,368


In [7]:
# Calculating the average number of transactions across all users
average_transactions_per_user = transactions_per_user.mean()

average_transactions_per_user

236.03688524590163

In [8]:
# Generate a unique account number
# Assuming account numbers are numeric, we'll find a number not present in the current dataset

existing_account_numbers = pd.to_numeric(data['Account No'], errors='coerce').dropna().unique()
new_account_number = int(max(existing_account_numbers) + 1)

# Replace null values in the "Account No" column with the new account number
data['Account No'].fillna(new_account_number, inplace=True)

# Verify that there are no longer null values in the "Account No" column
verification_result = data['Account No'].isnull().sum()

new_account_number, verification_result

(999752673, 0)

In [9]:
# Calculate the percentage of null values for each column
null_percentage = data.isnull().mean() * 100

null_percentage

Date                       0.099308
Timestamp                  0.108848
Account No                 0.000000
Balance                    0.106680
Amount                     0.090635
Third Party Account No    97.037243
Third Party Name           3.069871
dtype: float64

In [10]:
# Step 1: Drop the "Third Party Account No" column
data.drop(columns=['Third Party Account No'],inplace=True)
data.head()

,Date,Timestamp,Account No,Balance,Amount,Third Party Name
0,01/01/2023,00:00,678330503.0,2971.0,1584.0,Westport Care Home
1,01/01/2023,00:00,472213568.0,3792.0,1950.0,Barbiee Boutique
2,01/01/2023,00:00,472213568.0,3012.0,-780.0,NaN
3,01/01/2023,00:00,283027736.0,1787.0,780.0,NaN
4,01/01/2023,00:00,624500124.0,3226.0,1825.0,Fat Face


In [13]:


# Step 2: Handle null values in other columns
# For numerical columns (Balance, Amount), we'll use median imputation
# For categorical columns (Date, Timestamp, Account No, Third Party Name), we'll use the mode or a placeholder

numerical_columns = ['Balance', 'Amount']
categorical_columns = ['Date', 'Timestamp', 'Account No', 'Third Party Name']

# Fill numerical columns with their median
for col in numerical_columns:
    data[col].fillna(data[col].median(), inplace=True)

# Fill categorical columns with the mode or a placeholder
for col in categorical_columns:
    if col in ['Date', 'Timestamp', 'Account No']:  # These can be filled with mode
        mode_value = data[col].mode()[0]
        data[col].fillna(mode_value, inplace=True)
    else:  # "Third Party Name" can be filled with "Unknown"
        data[col].fillna('Unknown', inplace=True)

# Verify the manipulations by checking the new percentage of null values
new_null_percentage = data.isnull().mean() * 100

new_null_percentage


Date                0.0
Timestamp           0.0
Account No          0.0
Balance             0.0
Amount              0.0
Third Party Name    0.0
dtype: float64

In [15]:
data.isnull().sum()

Date                0
Timestamp           0
Account No          0
Balance             0
Amount              0
Third Party Name    0
dtype: int64